## Inspecting Data

In [1]:
import gen_input
import numpy as np
import os
from datetime import datetime
import tensorflow as tf

train_test_valid_split = [1., 0., 0.]

x_train, y_train = gen_input.read_data_sets("data/train_32x32.mat", train_test_valid_split).train.next_batch(100)
x_test, y_test = gen_input.read_data_sets("data/test_32x32.mat", train_test_valid_split).train.next_batch(100)

In [2]:
# Data information
input_channels = 3
image_size = 32
n_classes = 10

In [3]:
# Default stdev for weights and biases
init_std = 0.25
# Default decay, if non-zero
init_decay = 0.001

# We can't initialize these variables to 0 - the network will get stuck.
def weight_variable(shape, stddev=init_std):
  """Create a weight variable with appropriate initialization."""
  initial = tf.truncated_normal(shape, stddev)
  return tf.Variable(initial)

def bias_variable(shape, default_bias=init_std):
  """Create a bias variable with appropriate initialization."""
  initial = tf.constant(default_bias, shape=shape)
  return tf.Variable(initial)

def convlayer(layer_name, input_tensor, receptive_field, channels_in, channels_out,
              padding='SAME', stride=1, act=tf.nn.relu, decay=0,
              pool=False, pooler=tf.nn.max_pool, pool_size=2, pool_stride=2, pool_padding='SAME',
              batch_norm=True, training=True):
  """General purpose convolutional layer, followed by pooling

  It does a matrix convolution, bias add, and then uses relu by default to nonlinearize.
  Then it pools using max pooling by default.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops for TensorBoard.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([receptive_field, receptive_field, channels_in, channels_out])

      if decay > 0:
          weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
          tf.add_to_collection('losses', weight_decay)
  
    with tf.name_scope('biases'):
      biases = bias_variable([channels_out])
      
    with tf.name_scope('W_conv_x_plus_b'):
      preactivate = tf.nn.conv2d(input_tensor, weights, 
                                 strides=[1, stride, stride, 1], 
                                 padding=padding) + biases
    
    if batch_norm:
      with tf.name_scope('batchnorm'):
        normed = tf.layers.batch_normalization(preactivate, training=training)
      activations = act(normed, name='activation')
    else:
      activations = act(preactivate, name='activation')  
    
    if pool:
      with tf.name_scope('pool'):
        max_pool = pooler(activations, ksize=[1, pool_size, pool_size, 1], 
                        strides=[1, pool_stride, pool_stride, 1],
                        padding=pool_padding)
      return max_pool
    else: 
      return activations
    
def nn_layer(layer_name, input_tensor, input_dim, output_dim, act=tf.nn.relu, decay=0):
  """Reusable code for making a normal neural net layer.
  It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])
      
      if decay > 0:
        weight_decay = tf.multiply(tf.nn.l2_loss(weights), decay, name='weight_decay')
        tf.add_to_collection('losses', weight_decay)

    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
    activations = act(preactivate, name='activation')
    return activations
  
def flat_dimension(tensor):
  dim = 1 # Compute how many numbers we have, ignoring the batch size
  for d in tensor.get_shape()[1:].as_list():
    dim *= d
  return dim

# Normalize by subtracting per image, per channel means
def normalize_batch(batch):
  per_img_ch_means = batch.mean(axis=1)
  return batch - per_img_ch_means[:, np.newaxis, :]


In [4]:
def run(re_run=False):
  # RESET TF GRAPH, just in case
  tf.reset_default_graph()    
  sess = tf.InteractiveSession() ###### May need to change for .py files

  ts = datetime.now().strftime('%Y%m%d_%H%M')
  logs_path = "logs/{}/".format(ts)
  pwd = os.getcwd()+"/"
  print("tensorboard --logdir=/{}{}".format(pwd, logs_path))
  train_writer = tf.summary.FileWriter(logs_path + '/train', sess.graph)
  test_writer = tf.summary.FileWriter(logs_path + '/test')


  ### Place holders ###
  with tf.name_scope('test_train_variables'):
    training = tf.placeholder(tf.bool) # for batch_norm mode
    keep_prob = tf.placeholder(tf.float32) # for drop out
  #     Optionally track that place holders are correctly set at test and train tme
    tf.summary.scalar('training', tf.to_int32(training, name='ToInt32'))
    tf.summary.scalar('dropout_keep_probability', keep_prob)

  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, image_size*image_size, input_channels], name="x-input") 
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")
  
  ### Network ###
  with tf.name_scope('input_reshape'):
    input_reshaped = tf.reshape(x, [-1, image_size, image_size, input_channels])
    tf.summary.image('input', input_reshaped, 5) # Optionally save 5 images to ensure reshape is working

  conv1 = convlayer(layer_name='conv1', input_tensor=input_reshaped, receptive_field=5, 
                      channels_in=input_channels, channels_out=64, pool=True, pool_size=2, pool_stride=2,
                      batch_norm=False, training=training)
  print conv1.shape
  
  conv2 = convlayer(layer_name='conv2', input_tensor=conv1, receptive_field=5, 
                    channels_in=64, channels_out=64, pool=True, pool_size=2, pool_stride=2,
                    batch_norm=False, training=training)
  print conv2.shape
  
  with tf.name_scope('conv3_flatten'):
    conv_reshaped = tf.reshape(conv2, [-1, flat_dimension(conv2)])
  print conv_reshaped.shape
    
  fc1 = nn_layer(layer_name='fc1', input_tensor=conv_reshaped, input_dim=flat_dimension(conv2), output_dim=256, decay=init_decay)
  print fc1.shape
  
  # Do not apply softmax activation yet! use the identity
  y = nn_layer(layer_name='output', input_tensor=fc1, input_dim=256, output_dim=n_classes, act=tf.identity) 
  
  
  ### Operations ###
  # Cross-Entropy Loss
  with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
      tf.add_to_collection('losses', cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy)
  
  # Total loss (weight decay + cross-entropy)
  total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

  # Other metrics
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  # Merge all the summaries and write them out
  merged = tf.summary.merge_all()
  
  # Different options for running the graph
  def feed_dict(mode):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if mode == 'Train':
      batch_x, batch_y = x_train, y_train
      keep_proba = train_keep_prob
      training_ = True
    elif mode == 'Train_no_drop':
      batch_x, batch_y = x_train, y_train
      keep_proba = 1.0
      training_ = True
    elif mode == 'Test':
      batch_x, batch_y = x_test, y_test
      keep_proba = 1.0
      training_ = False
    batch_x = normalize_batch(batch_x) # Subtract per image mean
    return {x: batch_x, y_: batch_y, keep_prob: keep_proba, training: training_}

  # Might be needed for batch norm
  extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)  
  
  tf.global_variables_initializer().run()
  
  saver = tf.train.Saver()
  if re_run:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model{}.ckpt".format(run_number))
    print("Model {} restored".format(run_number))
    acc = sess.run(accuracy, feed_dict=feed_dict(mode='Train_no_drop'))
    print acc    
  
  if not(re_run):
    for epoch in xrange(training_epochs):
      for batch_num in xrange(total_batches):
        if batch_num % test_every == test_every - 1:  
          # Record summaries and accuracy on the *test* set
          summary = sess.run(merged, feed_dict=feed_dict(mode='Test'))
          test_writer.add_summary(summary, epoch * total_batches + batch_num)
          # Now proceed to train and produce corresponding training summary too
          summary, _, _ = sess.run([merged, train_step, extra_update_ops], feed_dict=feed_dict(mode='Train_no_drop'))
          train_writer.add_summary(summary, epoch * total_batches + batch_num)
        else:
          sess.run([train_step, extra_update_ops], feed_dict=feed_dict(mode='Train'))
    print "\nOptimization Finished!\n"

    save_path = saver.save(sess, "/tmp/model{}.ckpt".format(run_number))
    print("Model saved in file: %s" % save_path)

  
  train_writer.close()
  test_writer.close()

In [5]:
run_number = 0

In [6]:
learning_rate = 0.001
training_epochs = 100
train_keep_prob = 0.92

# Train batch size
batch_size = 20
total_batches = int(len(x_train) / batch_size)
print total_batches

# Test frequency / size
test_every = 5
test_batch_size = int(1*len(x_test))
print test_batch_size

5
100


In [7]:
# run(re_run=False)

In [8]:
run(re_run=True)

tensorboard --logdir=//home/andy/Desktop/cnn_svhn_old/logs/20170423_1715/
(?, 16, 16, 64)
(?, 8, 8, 64)
(?, 4096)
(?, 256)
INFO:tensorflow:Restoring parameters from /tmp/model0.ckpt
Model 0 restored
0.71
